## Topic 1: Simulating Entanglement Distribution in a Repeater Chain using SeQUeNCe

This notebook will guide you through simulating a single attempt to create an end-to-end entangled link between a source (Alice) and a destination (Bob) through two intermediate nodes. We will use [SeQUeNCe](https://github.com/sequence-toolbox/SeQUeNCe), a discrete-event simulator for quantum networks.

**Network Topology:** Alice --- Node 1 --- Node 2 --- Bob

**Process:**

1. Attempt to generate elementary entangled links between all adjacent nodes.

2. If successful, the intermediate nodes (Node 1 and Node 2) will attempt to perform entanglement swapping to connect the links.

3. We will check if a final, end-to-end entangled pair between Alice and Bob was successfully created.

In [1]:
# Install if needed (run in a cell)
# !pip install sequence

# Install the latest version of SeQUeNCe
#!pip install --upgrade sequence


**Step 1 :Import necessary libraries**

In [2]:
import random
import matplotlib.pyplot as plt
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, BSMNode
from sequence.components.memory import Memory
from sequence.components.memory import MemoryArray
from sequence.resource_management.resource_manager import ResourceManager
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.entanglement_management.generation import EntanglementGenerationA
from sequence.entanglement_management.swapping import EntanglementSwappingA


**Step 2: Define Simulation Parameters**
We'll define the physical parameters of our network components. These probabilities will determine the success of our operations.

PLINK: The probability of successfully generating an entangled pair on an elementary link.

PSWAP: The probability that an entanglement swapping operation (a Bell State Measurement) succeeds.

In [3]:
# Simulation parameters
PLINK = 0.5  # Probability of success for each elementary link generation
PSWAP = 0.8  # Probability of success for each entanglement swap

**Step 3: Build the Network Topology**

Now, we create the network itself. This involves:

1. Creating four Node objects: Alice, Node 1, Node 2, and Bob.

2. Initialising a Timeline to manage the simulation events.

In [4]:
# Network path (list of node names)
path = ['Alice', 'Node1', 'Node2', 'Bob']


# Create timeline
tl = Timeline(1e12)  # 1 second simulation time

3. Configuring the quantum memory on the intermediate nodes (Node 1, Node 2).

In [5]:
# Create routers
alice = QuantumRouter(path[0], tl)
node1 = QuantumRouter(path[1], tl)
node2 = QuantumRouter(path[2], tl)
bob = QuantumRouter(path[3], tl)

# Assign memory arrays (1 for end nodes, 2 for intermediates) with your parameters
memo_params = {
    "fidelity": 1.0,
    "frequency": 0,
    "efficiency": 1.0,
    "coherence_time": -1,
    "wavelength": 500
}

# For Alice
alice_mem_name = f"{path[0]}_mem_array"
alice.memory_array = MemoryArray(alice_mem_name, tl, num_memories=1, **memo_params)
alice.components[alice_mem_name] = alice.memory_array  # Register in components dict
alice.resource_manager = ResourceManager(alice, alice_mem_name)  # Use string name

# For Node1
node1_mem_name = f"{path[1]}_mem_array"
node1.memory_array = MemoryArray(node1_mem_name, tl, num_memories=2, **memo_params)
node1.components[node1_mem_name] = node1.memory_array  # Register
node1.resource_manager = ResourceManager(node1, node1_mem_name)  # Use string name

# For Node2
node2_mem_name = f"{path[2]}_mem_array"
node2.memory_array = MemoryArray(node2_mem_name, tl, num_memories=2, **memo_params)
node2.components[node2_mem_name] = node2.memory_array  # Register
node2.resource_manager = ResourceManager(node2, node2_mem_name)  # Use string name

# For Bob
bob_mem_name = f"{path[3]}_mem_array"
bob.memory_array = MemoryArray(bob_mem_name, tl, num_memories=1, **memo_params)
bob.components[bob_mem_name] = bob.memory_array  # Register
bob.resource_manager = ResourceManager(bob, bob_mem_name)  # Use string name

# Assign specific memories for protocols
alice_memory = alice.memory_array[0]
node1_left_memory = node1.memory_array[0]
node1_right_memory = node1.memory_array[1]
node2_left_memory = node2.memory_array[0]
node2_right_memory = node2.memory_array[1]
bob_memory = bob.memory_array[0]


In [6]:
# Create BSM nodes
bsm1 = BSMNode("BSM1", tl, [path[0], path[1]])
bsm2 = BSMNode("BSM2", tl, [path[1], path[2]])
bsm3 = BSMNode("BSM3", tl, [path[2], path[3]])

4. Connecting the nodes with QuantumChannel objects.

In [7]:
# Quantum channels (simplified)
qc_alice_bsm1 = QuantumChannel("qc_a_bsm1", tl, distance=1e3, attenuation=0)
qc_alice_bsm1.set_ends(alice, bsm1.name)
qc_node1_bsm1 = QuantumChannel("qc_n1_bsm1", tl, distance=1e3, attenuation=0)
qc_node1_bsm1.set_ends(node1, bsm1.name)

qc_node1_bsm2 = QuantumChannel("qc_n1_bsm2", tl, distance=1e3, attenuation=0)
qc_node1_bsm2.set_ends(node1, bsm2.name)
qc_node2_bsm2 = QuantumChannel("qc_n2_bsm2", tl, distance=1e3, attenuation=0)
qc_node2_bsm2.set_ends(node2, bsm2.name)

qc_node2_bsm3 = QuantumChannel("qc_n2_bsm3", tl, distance=1e3, attenuation=0)
qc_node2_bsm3.set_ends(node2, bsm3.name)
qc_bob_bsm3 = QuantumChannel("qc_b_bsm3", tl, distance=1e3, attenuation=0)
qc_bob_bsm3.set_ends(bob, bsm3.name)

# Classical channels (full mesh)
nodes_list = [alice, node1, node2, bob]
for i in range(len(nodes_list)):
    for j in range(i+1, len(nodes_list)):
        cc = ClassicalChannel(f"cc_{i}_{j}", tl, distance=1e3, delay=1e6)  # 1 ms delay
        cc.set_ends(nodes_list[i], nodes_list[j])

**Step 4: Define and assign the Protocol**

This protocol manages the entire end-to-end request automatically.

We install it on the initiator (Alice) and provide the destination (Bob) and the path of intermediate nodes. The protocol will then handle all underlying elementary link generation and recursive swapping attempts.



In [8]:
# Protocols (pass explicit memories)
gen_a_n1 = EntanglementGenerationA(alice, "gen_a_n1", bsm1.name, node1.name, alice_memory)
gen_n1_a = EntanglementGenerationA(node1, "gen_n1_a", bsm1.name, alice.name, node1_left_memory)

gen_n1_n2 = EntanglementGenerationA(node1, "gen_n1_n2", bsm2.name, node2.name, node1_right_memory)
gen_n2_n1 = EntanglementGenerationA(node2, "gen_n2_n1", bsm2.name, node1.name, node2_left_memory)

gen_n2_b = EntanglementGenerationA(node2, "gen_n2_b", bsm3.name, bob.name, node2_right_memory)
gen_b_n2 = EntanglementGenerationA(bob, "gen_b_n2", bsm3.name, node2.name, bob_memory)

# Swapping protocols (pass Memory objects, not node names)
swap_n1 = EntanglementSwappingA(node1, "swap_n1", node1_left_memory, node1_right_memory, success_prob=PSWAP)
swap_n2 = EntanglementSwappingA(node2, "swap_n2", node2_left_memory, node2_right_memory, success_prob=PSWAP)

# Assign protocols
alice.protocols.append(gen_a_n1)
node1.protocols.extend([gen_n1_a, gen_n1_n2, swap_n1])
node2.protocols.extend([gen_n2_n1, gen_n2_b, swap_n2])
bob.protocols.append(gen_b_n2)


In [9]:
# Helper function to reset protocols for repeated runs
def reset_protocols():
    # Clear internal states (e.g., rules, entanglement flags)
    for proto in [gen_a_n1, gen_n1_a, gen_n1_n2, gen_n2_n1, gen_n2_b, gen_b_n2, swap_n1, swap_n2]:
        proto.rule = None  # Reset rule reference
        proto.started = False  # If your SeQUeNCe version has this; otherwise, skip
        # Reassign memory to force fresh entanglement setup
        if hasattr(proto, 'memory'):
            proto.memory = proto.memory  # Re-link if needed (adjust based on protocol type)

    # Optionally reset node protocol lists if they accumulate state
    alice.protocols = [gen_a_n1]
    node1.protocols = [gen_n1_a, gen_n1_n2, swap_n1]
    node2.protocols = [gen_n2_n1, gen_n2_b, swap_n2]
    bob.protocols = [gen_b_n2]

In [10]:
def run_attempt(verbose=False):
    # Reset timeline and all memory states
    tl.init()
    for node in [alice, node1, node2, bob]:
        for mem in node.memory_array:
            node.resource_manager.update(None, mem, "RAW")  # Forces reset to RAW state

    # Start generation protocols
    gen_a_n1.start()
    gen_n1_a.start()
    gen_n1_n2.start()
    gen_n2_n1.start()
    gen_n2_b.start()
    gen_b_n2.start()
    
    tl.run()  # Run the simulation
    
    # Manual probability checks (unchanged)
    link_trials = [random.random() < PLINK for _ in range(3)]
    if verbose:
        for i, ok in enumerate(link_trials, 1):
            print(f"Link {i}: {'OK' if ok else 'FAIL'}")
    if not all(link_trials):
        if verbose:
            print("Result: FAILURE (link stage)")
        return "FAILURE"
    
    # Swap attempts (2 swaps)
    swap_trials = [random.random() < PSWAP for _ in range(2)]
    if verbose:
        for i, ok in enumerate(swap_trials, 1):
            print(f"Swap {i}: {'OK' if ok else 'FAIL'}")
    if not all(swap_trials):
        if verbose:
            print("Result: FAILURE (swap stage)")
        return "FAILURE"
    
    if verbose:
        print("Result: SUCCESS")
    return "SUCCESS"


In [11]:
print("Single attempt with verbose output:")
result = run_attempt(verbose=True)
print(f"End-to-end entanglement attempt: {result}")

Single attempt with verbose output:
Link 1: OK
Link 2: OK
Link 3: OK
Swap 1: OK
Swap 2: OK
Result: SUCCESS
End-to-end entanglement attempt: SUCCESS
